In [1]:
# pip install watermark
%load_ext watermark
%watermark -v -m -p numpy,scipy,pandas,matplotlib,sklearn -g 
#,statsmodels

CPython 3.6.7
IPython 7.4.0

numpy 1.16.2
scipy 1.2.1
pandas 0.24.2
matplotlib 3.0.3
sklearn 0.20.3

compiler   : GCC 8.2.0
system     : Linux
release    : 4.9.93-boot2docker
machine    : x86_64
processor  : x86_64
CPU cores  : 2
interpreter: 64bit
Git hash   :


In [2]:
from __future__ import division, print_function
import warnings
warnings.filterwarnings('ignore')
import os
import numpy as np
import pandas as pd
import math
import datetime
import re
import sys

In [92]:
COMPARATOR_AUTHOR = "Anton Tushev" 
COMPARATOR_VERSION = "v.11"
PATH_TO_DATA = '/notebooks/pinda/data'
#PATH_TO_STRUCG = '/notebooks/pinda/data/alcanc26_03_2019'
PATH_TO_STRUCG = '/notebooks/pinda/data/test'
PATH_TO_REPORTS = '/notebooks/pinda/reports'
FILE_NAME_DATA1 = 'TX04_fixed_v1.csv'
SD_RACK_NAMES = ['tx04']
FILE_NAME_TEMP1 = 'data_s1_@progress_bkup.csv'
FILE_NAME_TEMP2 = 'data_ls1_@progress_bkup.csv'
FILE_NAME_TEMP3 = 'data_lm_@progress_bkup.csv'
#FILE_NAME_DATA1 = 'test.csv'
IGNORE_CHART = ['@SIMD']
SKIP_EQUAL = True
SKIP_BLOCK_NEXST = True
SKIP_SIGNAL_NFND = True
SKIP_TYPE_STR = True
SKIP_CHART_NFND = True
COMPARATOR_README = "===================================================================================\
\r\nAdvanced comparator SimadynD <=> TDC software version: %s \r\nAuthor: %s \
\r\nComparison result made from %s StrucG folder \
\r\nTDC data file is '%s' \
\r\nList of SimadynD subfolders for parsing is %s \
\r\nList of ignored charts is %s \
\r\n===================================================================================\
"%(COMPARATOR_VERSION,COMPARATOR_AUTHOR,PATH_TO_STRUCG,FILE_NAME_DATA1,SD_RACK_NAMES,IGNORE_CHART)
print(COMPARATOR_README)

Advanced comparator SimadynD <=> TDC software version: v.11 
Author: Anton Tushev 
Comparison result made from /notebooks/pinda/data/test StrucG folder 
TDC data file is 'TX04_fixed_v1.csv' 
List of SimadynD subfolders for parsing is ['tx04'] 
List of ignored charts is ['@SIMD'] 


In [85]:
#Load TDC data
data_a1 = pd.read_csv(os.path.join(PATH_TO_DATA,FILE_NAME_DATA1),keep_default_na=False,na_values=[''], encoding="cp1251",sep=';',header=None)
data_a1 = data_a1[[0,1,3,5,11,13,28,29]]
data_a1.columns = ['rack_cpu','chart','block','signal','value','link','type','out']
data_a1 = pd.concat([data_a1, data_a1['rack_cpu'].str.split("\\",expand = True)[[0,1]]], axis=1) 
data_a1.columns = ['rack_cpu','chart','block','signal','value','link','type','out','rack','cpu']
data_a1["out"]= data_a1["out"].replace(("IN","OUT"),(0,1)).astype('bool') 
data_a1 = data_a1[~data_a1['chart'].isin(IGNORE_CHART)]
data_a1["block"] = data_a1["block"].astype(str)
data_a1["chartblock"] = data_a1["chart"].str.upper()+"/"+data_a1["block"].str.upper()
print("Data types ",data_a1['type'].unique())
print("Charts ",data_a1['chart'].unique())
data_a1.head()

Data types  ['BOOL' 'REAL' 'SDTIME' 'WORD' 'INT' 'DINT' 'GLOBAL' 'STRING' 'DWORD'
 'BYTE']
Charts  ['ENCC_1' 'ENBRU1' 'ENBRI1' 'ENBEA1' 'BGT__1' '@SEND1' 'ENGAU1' 'ENGN11'
 'ENMB_1' 'ENOSW1' 'ENGUA1' 'ENHDR1' 'ENIHW1' 'ENINS1' 'ENEXT1' 'ENENT1'
 'ENSR_1' 'ENSPH1' 'ENSCD1' 'ENPIN1' 'ENPEL1' 'ENPAR1' 'ENISW1' 'ENLAP1'
 'ENMA_1' 'ENMAN1' 'ENOHW1' 'ENSTR1' 'ENTEM1' 'ENTUR1' 'MESY_1' 'MONIM1'
 'MONIW1' 'SERV_1' 'VISU_1' 'WPS__1' 'ENISW_MSU' 'ENDIA1' 'ENCRA1'
 'ENCPT1' 'ENCPS1' 'ENCPP1' 'ENCPM1' 'ENCPH1' 'ENCPC1' 'ENCPA1' 'ENCOO1'
 'ENMG_1' 'ENMC_1' 'ENMD_1' 'ENME_1' 'ENMF_1' '@SEND2' 'EXBAN2' 'EXBBH2'
 'EXBBY2' 'EXBEA2' 'EXBW_2' 'EXBWA2' 'EXBWT2' 'EXCC_2' 'EXCRA2' 'EXCRM2'
 'EXCUT2' 'EXDEF2' 'EXDIA2' 'EXFEL2' 'EXGAU2' 'EXGN12' 'EXGN22' 'EXIHW2'
 'EXIR_2' 'EXIRP2' 'EXISW2' 'EXKNA2' 'EXLAP2' 'EXMAN2' 'EXMLR2' 'EXMLT2'
 'EXOHW2' 'EXOSW2' 'EXPAR2' 'EXPUS2' 'EXREJ2' 'EXREL2' 'EXSC_2' 'EXSCA2'
 'EXSCM2' 'EXSH12' 'EXSH22' 'EXSHJ2' 'EXSHT2' 'EXSML2' 'EXSR_2' 'EXSRC2'
 'EXSRE2' 'EXSSC2' 'EXSSR2' 'EX

,rack_cpu,chart,block,signal,value,link,type,out,rack,cpu,chartblock
0,TX04\D01P01\,ENCC_1,ABLOKD,I1,NaN,ENCC_1\AND209.Q,BOOL,False,TX04,D01P01,ENCC_1/ABLOKD
1,TX04\D01P01\,ENCC_1,ABLOKD,I2,NaN,ENCC_1\NOTBDW.Q,BOOL,False,TX04,D01P01,ENCC_1/ABLOKD
2,TX04\D01P01\,ENCC_1,ABLOKD,Q,0,ENCC_1\XN1030.I4,BOOL,True,TX04,D01P01,ENCC_1/ABLOKD
3,TX04\D01P01\,ENCC_1,ADD100,X1,NaN,ENCC_1\DIV001.Y,REAL,False,TX04,D01P01,ENCC_1/ADD100
4,TX04\D01P01\,ENCC_1,ADD100,X2,NaN,ENCC_1\CTB003.Y15,REAL,False,TX04,D01P01,ENCC_1/ADD100


In [87]:
#Generate Simadyn chart structure
def sd_get_chart_list(PATH_TO_STRUCG=PATH_TO_STRUCG,SD_RACK_NAMES=SD_RACK_NAMES):
    #filter chart files
    n = re.compile("^(((?!old).)*).((cfp)|(ofp))$")
    file_list=[] 
    file_list = {rack_name:[item for item in os.listdir(os.path.join(PATH_TO_STRUCG,rack_name)) \
        # chart file names filter conditions
        if os.path.isfile(os.path.join(PATH_TO_STRUCG,rack_name, item)) and (n.search(item)) and len(item)<11 \
        ] for rack_name in SD_RACK_NAMES}
    file_list_filtered = dict(file_list)
    for i in file_list:
        for k in file_list[i]:
            chart_name = k.split('.')[0]
            if "%s.cfp"%chart_name in file_list[i]:
                try:
                    file_list_filtered[i].remove("%s.ofp"%chart_name)
                except ValueError:
                    pass  # do nothing
    out_dict = dict()
    for i in file_list_filtered:
        out_dict[i] = dict()
        for k in file_list_filtered[i]:
            out_dict[i][k.split('.')[0]]=os.path.join(PATH_TO_STRUCG,i,k)
    if out_dict:
        return out_dict
    else:
        return -1
    
#Generate Simadyn cpu mapping structure
def sd_get_cpu_charts_list(PATH_TO_STRUCG=PATH_TO_STRUCG,SD_RACK_NAMES=SD_RACK_NAMES):
    #filter chart files
    n = re.compile("^((\d+).*)\.(mpn)$")
    file_list=[] 
    file_list=[] 
    file_list = {rack_name:[item for item in os.listdir(os.path.join(PATH_TO_STRUCG,rack_name)) \
        # chart file names filter conditions
        if os.path.isfile(os.path.join(PATH_TO_STRUCG,rack_name, item)) and (n.search(item)) and len(item)<11 \
        ] for rack_name in SD_RACK_NAMES}
    out_dict = dict()
    for i in file_list:
        out_dict[i] = dict()
        for k in file_list[i]:
            out_dict[i][k.split('.')[0]]=os.path.join(PATH_TO_STRUCG,i,k)
    if out_dict:
        return out_dict
    else:
        return -1
    
sd_chart_list = sd_get_chart_list() 
sd_cpu_charts_list = sd_get_cpu_charts_list() 

##output => #sd_chart_list {"tx03":{'chrt_name':file_addrs}}

In [11]:
#n = re.search("^(((?!old).)*).((cfp)|(ofp))$", 'pray1___20131208.cfp')
test_txt = []
test_txt.append(" 325	I1  B1 < VIHW09.Q15&")
test_txt.append('  81	IS  V2 < $VISW17')
test_txt.append(" 163	Q1  B1 > ,'Man__CPS'            \"MAINTENANCE COIL PREP STATION\"")
test_txt.append(' 115	I1  B1 < VIHW17.Q2&')
test_txt.append(' 519	X1  N2 < @TYP=V2,0HFFFF')
test_txt.append(' 520	X2  N2 < @TYP=V2,LB2.QS&')
test_txt.append(' 527	QS  V2 > $VGAU02                "Diagnose 2nd direction"')
test_txt.append(' 528	Q   B1 > ')
test_txt.append("  31	AD  NK = D02_I1.X8C")
test_txt.append("  32	DM  B1 - 1")
test_txt.append("  22	AR  NS - 'B1G1X1'")
test_txt.append("  21	CTS CR - D0300A")
test_txt.append("1151	T1  NS - '@F@85@ EN QFC Coil P. Shear Tab. Air Float'")
test_txt.append("1199	X1  N2 < 0%")
test_txt.append("1200	X2  N2 < SEND07.QS")
test_txt.append("1201	I   B1 < EN_MES.Q,'EN_MES'&")
test_txt.append(" 566	LZU NF < #10000[ms],'RunTime '  \"Run time monitoring\"")
test_txt.append("1207	X2  N2 < 18.298%                \"18,3 representa 3000 kg\"")
test_txt.append("1283	X2  NF < #0")
test_txt.append("1304	X2  NF < #26,'GRAUS'")
test_txt.append(" 905	X2  NF < #100.0000000E3")
test_txt.append("1074	Y   NF > @TYP=TF,$PTTI_A,'PTTI_AUT'&")
test_txt.append("1073	X   NF < @TYP=TF,10[S]")
test_txt.append("2983	X2  NF < #530.0000000E-3,'m'")
test_txt.append(" 439	FBU NF < #500[ms],'Feedback'    \"Feedback monitoring\"")
test_txt.append(" 494	I   B1 < STG641.Q")
test_txt.append(" 495	Y   N2 > @TYP=V2")
test_txt.append("  17	X1  NF < $XIHW05,SCAL=1[m]")
test_txt.append(" 374	T   TF <  3           [s]")
test_txt.append(" 169	T   TF <            2[s ]")
test_txt.append("2946	OR  V2 < 0B00000000 00000000")
test_txt.append(" 108	X05 NF < #2500.0[V/(m/s)],'ky'  \"Scaling factor for control output\"&")
test_txt.append("2756	CRT TR = !EXM504")
test_txt.append(" 337	Q1  B1 >")
test_txt.append("1407	T   TF < #2.5 [s]")
test_txt.append("  87	Q3  B1 > $E_STOP PN,'E_STOP'&")

test_txt2 = " 439	FBU NF < #500[ms],'Feedback'    \"Feedback monitoring\""


#m = re.compile(r"^\s*\d+\s+(\w+)\s+(\w+)\s+([<>])\s+(@TYP=(..),)?([^@\"',&]+)([,\"'&]|$)", )
m = re.compile(r"""
        ^\s*\d+\s+ #string number
        (\w+)\s+   #signal name 
        (\w+)\s+   #signal type 
        ([<>=-])\s*  #signal delimiter 
        (@TYP=(..),?)? #singal type
        (   #value
        ([a-zA-Z0-9_.]+)|  #can be connection to other block
        ([$%!\[\]/()a-zA-Z0-9_.#-]+)|  #can be value
        ([$a-zA-Z0-9_]+\s*[$a-zA-Z0-9_]*)|  #can be virtual connection
        ([#0-9.]+\s*\[.*\])|  #can be time with spaces
        ([0-9B]+\s+[0-9]+)|  #can be hex in bit representation
        ('.*') #can be text
        )? #end of value
        (,)? #divider
        ('.*')? #link comment
        (,.*)? #some comment
        (\s+".*")? #signal comment
        (&)? #check next string
        ($) #end of string
        """, re.X)
k = re.compile(r"^\s*\d+\s+(\w+)\s+(\w+)\s+([<>=-])")

for a in test_txt:
    n = m.search(a)
    print(a)
    #print(n.group(1),n.group(2),n.group(3),n.group(5),n.group(6),n.group(13))
    print(n.group(6))
    #print(n)
    
#n = m.search(test_txt2)
#print("OUT:",n.group(1),n.group(2),n.group(3),n.group(6),n.group(13))

#for a in test_txt:
#    n = k.search(a)
#    print(n)

#for a in test_txt:
#    n = k.search(a)
#    print(n)

 325	I1  B1 < VIHW09.Q15&
VIHW09.Q15
  81	IS  V2 < $VISW17
$VISW17
 163	Q1  B1 > ,'Man__CPS'            "MAINTENANCE COIL PREP STATION"
None
 115	I1  B1 < VIHW17.Q2&
VIHW17.Q2
 519	X1  N2 < @TYP=V2,0HFFFF
0HFFFF
 520	X2  N2 < @TYP=V2,LB2.QS&
LB2.QS
 527	QS  V2 > $VGAU02                "Diagnose 2nd direction"
$VGAU02
 528	Q   B1 > 
None
  31	AD  NK = D02_I1.X8C
D02_I1.X8C
  32	DM  B1 - 1
1
  22	AR  NS - 'B1G1X1'
'B1G1X1'
  21	CTS CR - D0300A
D0300A
1151	T1  NS - '@F@85@ EN QFC Coil P. Shear Tab. Air Float'
'@F@85@ EN QFC Coil P. Shear Tab. Air Float'
1199	X1  N2 < 0%
0%
1200	X2  N2 < SEND07.QS
SEND07.QS
1201	I   B1 < EN_MES.Q,'EN_MES'&
EN_MES.Q
 566	LZU NF < #10000[ms],'RunTime '  "Run time monitoring"
#10000[ms]
1207	X2  N2 < 18.298%                "18,3 representa 3000 kg"
18.298%
1283	X2  NF < #0
#0
1304	X2  NF < #26,'GRAUS'
#26
 905	X2  NF < #100.0000000E3
#100.0000000E3
1074	Y   NF > @TYP=TF,$PTTI_A,'PTTI_AUT'&
$PTTI_A
1073	X   NF < @TYP=TF,10[S]
10[S]
2983	X2  NF < #530.0000000E-

In [229]:
#test_txt="   &	V4.2.6 FP-COP CHK 22.02.;9 07:54  (MP-TX04.PN-D05_P2 13.09.;6 05:02)"
#test_txt = "   1 PN-D01_P1"
#test_txt = "1832	*"
#test_txt = ""
#test_txt = "  89 Connector list of the function packets"
#test_txt = "1816	FP-@SEND1,COM"
#test_txt = "1679	   $VINS02 T4 < PN     PN-D01_P1"
test_txt = " 830	   $DIADE  T4 < PN     PN-D01_P1"

#cp = re.compile(r'^\s+\d+\s+([a-zA-Z0-9_-]+)$')
#cp = re.compile(r'^\s+\d+\s+Connector list of the function packets')
#cp = re.compile(r'^\s*\d+\s+\*')
#cp = re.compile(r'^\s*\d+\s+FP-([a-zA-Z0-9_\-@]+)')
cp = re.compile(r'^\s*\d+\s+(\$[a-zA-Z0-9_\-@]+)\s+([\w]+)\s((<)|(>))\s([\w]+)\s+([a-zA-Z0-9_-]+)$')
cp_s = cp.search(test_txt)
print(cp_s)
print(cp_s.group(1),cp_s.group(3),cp_s.group(7))

<_sre.SRE_Match object; span=(0, 37), match=' 830\t   $DIADE  T4 < PN     PN-D01_P1'>
$DIADE < PN-D01_P1


In [123]:
test_txt = []
test_txt.append("   &		(SEND01.I9)")
test_txt.append('   &		(KA1014.I2,KA1015.I2)')
k = re.compile(r"^\s*&+\s+\((.*)\)\s*$")
for a in test_txt:
    n = k.search(a)
    print(n.group(1))
    #print(n)

SEND01.I9
KA1014.I2,KA1015.I2


In [89]:
#########################row data generating library#################
NAME,TYPE_BASE,INOUT,TYPE_CONV,VALUE,CPU,CPU_SOURCE,CPU_LINK,CHART,BLOCK, LINKS  = 0,1,2,3,4,5,6,7,8,9,10

#open chart for reading
def sd_open_chart(chart_addr):
    try:
        with open(chart_addr) as f:
            lines = [line.rstrip('\n') for line in f]
    except EnvironmentError: # parent of IOError, OSError *and* WindowsError where available
        #print('ERROR File not exists!')
        return -1
    return lines

#generate chart mapping: line {block name:[block line begin, block line end]}
def sd_get_chart_map(inp_line):
    b = re.compile(r"^\s*\d+ ?([a-zA-Z0-9_]+) +: [a-zA-Z0-9_.@]+\s*,\s*POS=")
    el = re.compile(r'^ *\d+\s+[+]')
    #cp = re.compile(r'\(([a-zA-Z0-9_-]+).([a-zA-Z0-9_-]+)\s+\S+\s+\S+\)') #not used, because not all charts consist
    block_list = []
    mark_bloc = False
    #mark_cpu = False
    out_dict = dict()
    line_counter = 0
    for k in inp_line:
        #Find rack name and cpu name, e.g. MP-TX04 PN-D05_P2
        #if not mark_cpu:
        #    cp_s = cp.search(k)
        #    if cp_s :
        #        rack,cpu = cp_s.group(1),cp_s.group(2)
        #        mark_cpu = True
        b_s = b.search(k)
        if b_s :
            mark_bloc = True
            curr_block_name = b_s.group(1)
            block_list.append(curr_block_name)
            mark_bloc_line_beg = line_counter
        elif el.search(k):
            if mark_bloc:
                mark_bloc = False
                out_dict[curr_block_name] = [mark_bloc_line_beg,line_counter]
        line_counter = line_counter+1        
    #print(len(block_list))
    if(out_dict):
        return out_dict
    else:
        return -1

#check if line is signal or link
def sd_check_line_signal(line):
    sf = re.compile(r"^\s*\d+\s+(\w+)\s+(\w+)\s+([<>=-])")
    sa = re.compile(r"^\s*&+\s+(\(.*\))\s*$")
    if sf.search(line):
        out = 1 #signal
    elif sa.search(line):
        out = 2 #link
    else:
        out = 0 #nothing
    return out

#extract data from signal line
def sd_get_line_data(line): 
    m = re.compile(r"""
        ^\s*\d+\s+ #string number
        (\w+)\s+   #signal name 
        (\w+)\s+   #signal type 
        ([<>=-])\s*  #signal delimiter 
        (@TYP=(..),?)? #singal type
        (   #value
        ([a-zA-Z0-9_.]+)|  #can be connection to other block
        ([$%!\[\]/()a-zA-Z0-9_.#-]+)|  #can be value
        ([$a-zA-Z0-9_]+\s*[$a-zA-Z0-9_]*)|  #can be virtual connection
        ([#0-9.]+\s*\[.*\])|  #can be time with spaces
        ([0-9B]+\s+[0-9]+)|  #can be hex in bit representation
        ('.*') #can be text
        )? #end of value
        (,)? #divider
        ('.*')? #link comment
        (,.*)? #some comment
        (\s+".*")? #signal comment
        (&)? #check next string
        ($) #end of string
        """, re.X)
    #print(line)
    n = m.search(line)
    try: 
        out = list([n.group(1),n.group(2),n.group(3),n.group(5),n.group(6)])
    except:
        print(line)
        raise
        
                 #name      type       dir        type_conv  value      
    return list([n.group(1),n.group(2),n.group(3),n.group(5),n.group(6)])

#exstract link from like
def sd_get_line_link(line): 
    m = re.compile(r"^\s*&+\s+\((.*)\)\s*$")
    n = m.search(line)     
    return n.group(1)  

#get block data: signals, values, links, data types...
def sd_get_block_data(lines,chart,block,data_link_map_df):

    mark_signal_row_name = None
    mark_signal_row_data = list([[],[]]) #[[signal data],[links]]
    mark_block_row_data = []
    count = 0;
    for l in lines:
        line_check_result = sd_check_line_signal(l)
        if line_check_result:
            if line_check_result==1:  
                #if found a signal line
                count = count+1
                #add previous signal to preparation table
                if mark_signal_row_name:
                    #print("Old name:",mark_signal_row_name)
                    mark_block_row_data.append(mark_signal_row_data)
                    mark_signal_row_data = list([[],[]]) #reset buffer table
                #NAME,TYPE_BASE,INOUT,TYPE_CONV,VALUE
                mark_signal_row_data[0] = sd_get_line_data(l)
                #get CPU
                cpu = sd_get_chart_cpu(data_link_map_df,chart)
                mark_signal_row_data[0].append(cpu)
                #If value is $LINK here is a logic:
                #Copy value $LINK to link column
                if re.search(r'^\$',str(mark_signal_row_data[0][VALUE])) and mark_signal_row_data[0][INOUT]=="<":
                    
                    link = mark_signal_row_data[0][VALUE]
                    mark_signal_row_data[1].extend([link])
                    #get CPU_SOURCE
                    cpu_source = sd_get_block_link_source_cpu(data_link_map_df,mark_signal_row_data[0][VALUE],chart)
                    mark_signal_row_data[0].append(cpu_source)
                    #get CPU_LINK   (means CPU_SOURCE_LINK)
                    mark_signal_row_data[0].append(cpu_source+"_"+link)
                else:
                    #get CPU_SOURCE
                    mark_signal_row_data[0].append(cpu)
                    #get CPU_LINK   (means CPU_SOURCE_LINK)
                    mark_signal_row_data[0].append(None)
                    
                mark_signal_row_name = mark_signal_row_data[0][NAME]
                #get CHART,BLOCK
                mark_signal_row_data[0].append(chart)
                mark_signal_row_data[0].append(block)
            elif line_check_result==2:
                #if found a link line: (SOMETHING,SOMETHING2)
                mark_signal_row_data[1].extend(sd_get_line_link(l).split(','))
            #print(mark_signal_row_data)            
        else:
            pass #skip string
    #Add last signal to preparation storage
    mark_block_row_data.append(mark_signal_row_data)
    return mark_block_row_data

#convert list to pandas data frame
def sd_conv_block_data_to_df(block_data):
    #list element of row signal data:[[NAME,TYPE_BASE,INOUT,TYPE_CONV,VALUE],[links]]
    block_df = pd.DataFrame([[j for j in k[0]]+[k[1]] for k in block_data])
    return block_df
#convert list to better representation
def sd_conv_block_data_to_list(block_data):
    #list element of row signal data:[[NAME,TYPE_BASE,INOUT,TYPE_CONV,VALUE,CPU,CPU_SOURCE,CPU_LINK,CHART,BLOCK],[links]]
    block_data_list = [[j for j in k[0]]+[k[1]] for k in block_data]
    return block_data_list


#create df row with link source structure: chart_block_signal,cpu_link
def sd_get_source_link(block_data_list):
    #block_data_list: (NAME,TYPE_BASE,INOUT,TYPE_CONV,VALUE,CPU,CPU_SOURCE,CPU_LINK,CHART,BLOCK ),LINKS
    m = re.compile(r"^\$\w+")
    out_df = []
    for j in block_data_list:
        if j[INOUT]==">":
            if(j[INOUT]):
                val = str(j[VALUE])
                n = m.search(val) 
                if n:
                    #chart\block.signal,cpu_link
                    chart_block_signal = j[CHART].upper()+'\\'+j[BLOCK].upper()+'.'+j[NAME].upper()
                    cpu_link = j[CPU]+'_'+val
                    #print(chart_block_signal,"=>",cpu_link)
                    out_df.append([chart_block_signal,cpu_link])
            #look for source link in link array
            for k in j[LINKS]:
                if(k):
                    n = m.search(str(k)) 
                    if n:
                        #chart\block.signal,cpu_link
                        chart_block_signal = j[CHART].upper()+'\\'+j[BLOCK].upper()+'.'+j[NAME].upper()
                        cpu_link = j[CPU]+'_'+str(k)
                        #print(chart_block_signal,"=>",cpu_link)
                        out_df.append([chart_block_signal,cpu_link])
    return out_df

#generate data of all charts and generate data of link sources
def sd_build_data_s1(sd_chart_list,data_link_map_df):
    
    data_list = []
    data_link_source_list = []
    for i in sd_chart_list:
        for (k,v) in sd_chart_list[i].items(): 
            chart = k.upper()
            chart_lines = sd_open_chart(v)
            chart_mapping = sd_get_chart_map(chart_lines)
            print("Parsing '%s' chart with %s functions"%(chart,len(chart_mapping)))
            for block_map in chart_mapping:
                #generating array 'signal','type_base','inout','type_conv','value','link','chart','block','cpu'
                block_lines = chart_lines[chart_mapping[block_map][0]:chart_mapping[block_map][1]]
                #get block data: NAME,TYPE_BASE,INOUT,TYPE_CONV,VALUE,CPU,CPU_SOURCE,CPU_LINK,CHART,BLOCK
                block_data = sd_get_block_data(block_lines,chart,block_map,data_link_map_df)
                #print("before ",block_data)
                #print("-- ")
                block_data_list = sd_conv_block_data_to_list(block_data)
                #print("after ",block_data_list)
                #print("-- ")
                #Add block data to accumulation list
                data_list.extend(block_data_list)
                #create link source array
                block_link_source_list = sd_get_source_link(block_data_list)
                if len(block_link_source_list)!=0:
                    #print(block_link_source_list)
                    data_link_source_list.extend(block_link_source_list) 
    #Convert to DataFrame for easy work
    data_link_source_df = pd.DataFrame(data_link_source_list, columns=['chart_block_signal','cpu_link'])
    data_df = pd.DataFrame(data_list, columns=['signal','type_base','inout','type_conv','value','cpu',\
                                                         'cpu_source','cpu_link','chart','block','link'])
    return data_df,data_link_source_df

#parsing mapping file
#generate cpu,link db by cpu mapping file
def sd_build_cpu_map(sd_chart_list):
    data_link_map_list = []
    cp = re.compile(r'^\s+\d+\s+([a-zA-Z0-9_-]+)$') #find CPU name in file
    bg = re.compile(r'^\s+\d+\s+Connector list of the function packets') # find beginning of link information
    en = re.compile(r'^\s*\d+\s+\*')
    ch = re.compile(r'^\s*\d+\s+FP-([a-zA-Z0-9_\-@]+)')
    lk = re.compile(r'^\s*\d+\s+(\$[a-zA-Z0-9_\-@]+)\s+([\w]+)\s((<)|(>))\s([\w]+)\s+([a-zA-Z0-9_-]+)$')
    
    for i in sd_chart_list:
        for (k,v) in sd_chart_list[i].items():
            chart_lines = sd_open_chart(v)
            print("Parsing '%s' chart "%(k))
            cpu_mark = False
            beg_mark = False
            end_mark = False
            for l in chart_lines:
                ##Find CPU name
                if not cpu_mark:
                    cp_s = cp.search(l)
                    if cp_s :
                        cpu = cp_s.group(1)
                        cpu_mark = True
                        print(cpu)   
                else:
                    if not beg_mark:
                        if bg.search(l):
                            beg_mark = True
                    else:
                        #check end
                        if en.search(l):
                            end_mark = True
                            break
                        else:
                            #main block:
                            ch_s = ch.search(l)
                            if ch.search(l): #if chart beginning 1739	FP-MONIW1
                                chart = ch_s.group(1)
                            else:
                                lk_s = lk.search(l)
                                if lk_s: #if chart link definition 1744	   $DMALB1 T5 < PN     PN-D01_P1
                                    #'cpu','chart','out','link','cpu_source'
                                    data_link_map_list.append([cpu, chart.upper(),lk_s.group(3),lk_s.group(1),lk_s.group(7)])
    data_link_map_df = pd.DataFrame(data_link_map_list, columns=['cpu','chart','out','link','cpu_source'])
    return data_link_map_df

def sd_get_chart_cpu(data_link_map_df,chart):
    return data_link_map_df[data_link_map_df['chart']==chart].iloc[0]['cpu']

def sd_get_block_link_source_cpu(data_link_map_df,link,chart):
    return data_link_map_df[(data_link_map_df['chart']==chart)&\
                            (data_link_map_df['link']==link)&\
                            (data_link_map_df['out']=="<")].iloc[0]['cpu']

def sd_get_link_partner(data_link_source_df,cpu_link):
    try:
        return data_link_source_df[(data_link_source_df['cpu_link']==cpu_link)].iloc[0]['chart_block_signal']
    except IndexError:
        return None
    

In [90]:
#execution cell
#creating row data DataFrame "data_s1" from chart list
        
data_lm = sd_build_cpu_map(sd_cpu_charts_list)
data_s1,data_ls1 = sd_build_data_s1(sd_chart_list,data_lm)
data_s1["block"] = data_s1["block"].astype(str)

data_s1

Parsing '1tx04' chart 
PN-D01_P1
Parsing '2tx04' chart 
PN-D05_P2
Parsing '3tx04' chart 
PN-D08_P3
Parsing 'MONIW1' chart with 52 functions
Parsing 'MONIW2' chart with 52 functions
Parsing 'SERV_1' chart with 12 functions
Parsing 'SERV_2' chart with 12 functions
Parsing 'VISU_1' chart with 177 functions
Parsing 'VISU_2' chart with 122 functions
Parsing 'WPS__1' chart with 12 functions
Parsing 'WPS__2' chart with 12 functions
Parsing 'EXMLT2' chart with 50 functions
Parsing 'EXOHW2' chart with 218 functions
Parsing 'EXOSW2' chart with 368 functions
Parsing 'EXPAR2' chart with 3 functions
Parsing 'EXPUS2' chart with 222 functions
Parsing 'EXREJ2' chart with 48 functions
Parsing 'EXREL2' chart with 47 functions
Parsing 'EXSCA2' chart with 74 functions
Parsing 'EXSCM2' chart with 44 functions
Parsing 'EXSC_2' chart with 62 functions
Parsing 'EXSH12' chart with 83 functions
Parsing 'EXSH22' chart with 72 functions
Parsing 'EXSHJ2' chart with 83 functions
Parsing 'EXSHT2' chart with 58 funct

,signal,type_base,inout,type_conv,value,cpu,cpu_source,cpu_link,chart,block,link
0,I,B1,<,None,1,PN-D01_P1,PN-D01_P1,None,MONIW1,ENMESS,[]
1,T,TF,<,None,60[s],PN-D01_P1,PN-D01_P1,None,MONIW1,ENMESS,[]
2,Q,B1,>,None,None,PN-D01_P1,PN-D01_P1,None,MONIW1,ENMESS,"[NSW003.I, NSW000.I, NSW001.I, NSW002.I]"
3,X1,N2,<,V2,0H0,PN-D01_P1,PN-D01_P1,None,MONIW1,NSW000,[]
4,X2,N2,<,V2,$VIHW36,PN-D01_P1,PN-D01_P1,PN-D01_P1_$VIHW36,MONIW1,NSW000,[$VIHW36]
5,I,B1,<,None,ENMESS.Q,PN-D01_P1,PN-D01_P1,None,MONIW1,NSW000,[]
6,Y,N2,>,V2,None,PN-D01_P1,PN-D01_P1,None,MONIW1,NSW000,[VIHW36.IS]
7,X1,N2,<,V2,0H0,PN-D01_P1,PN-D01_P1,None,MONIW1,NSW001,[]
8,X2,N2,<,V2,$VOHW01,PN-D01_P1,PN-D01_P1,PN-D01_P1_$VOHW01,MONIW1,NSW001,[$VOHW01]
9,I,B1,<,None,ENMESS.Q,PN-D01_P1,PN-D01_P1,None,MONIW1,NSW001,[]


In [93]:
#save intermidiate result
data_s1.to_csv(os.path.join(PATH_TO_DATA,FILE_NAME_TEMP1))
data_ls1.to_csv(os.path.join(PATH_TO_DATA,FILE_NAME_TEMP2))
data_lm.to_csv(os.path.join(PATH_TO_DATA,FILE_NAME_TEMP3))


In [ ]:
#restore intermediate result
data_s1 = pd.read_csv(os.path.join(PATH_TO_DATA,FILE_NAME_TEMP1))
data_ls1 = pd.read_csv(os.path.join(PATH_TO_DATA,FILE_NAME_TEMP2))
data_lm = pd.read_csv(os.path.join(PATH_TO_DATA,FILE_NAME_TEMP3))

In [94]:
data_ls1

,chart_block_signal,cpu_link
0,SERV_1/SERHMI.QTS,PN-D01_P1_$BSER02
1,SERV_1/SERL01.QTS,PN-D01_P1_$BLOC1Q
2,SERV_1/SERL02.QTS,PN-D01_P1_$BLOC2Q
3,SERV_1/SERL03.QTS,PN-D01_P1_$BLOC3Q
4,SERV_1/MSI_VI.QTS,PN-D01_P1_$BHMI1Q
5,SERV_1/MSI_PR.QTS,PN-D01_P1_$BPRI1Q
6,SERV_2/SERHMI.QTS,PN-D05_P2_$BSER02
7,SERV_2/SERL01.QTS,PN-D05_P2_$BLOC1Q
8,SERV_2/SERL02.QTS,PN-D05_P2_$BLOC2Q
9,SERV_2/SERL03.QTS,PN-D05_P2_$BLOC3Q


In [80]:
#find partnet link
def sd_rect_data_s1(data_df,data_link_source_df):
    def lookup(x):
        partner = sd_get_link_partner(data_link_source_df,x)
        return partner
    data_df['partner'] = data_df['cpu_link'].apply(lookup)
    return data_df

#rectify function: add block "chartblock"
def sd_rect_data_s2(data_df):
    data_df['chartblock'] = data_df["chart"].str.upper()+"/"+data_df["block"].str.upper()
    return data_df



In [95]:
data_s2 = sd_rect_data_s1(data_s1.copy(),data_ls1)

In [96]:
data_s2.loc[data_s2['partner']!=None,'partner']

0                     None
1                     None
2                     None
3                     None
4         ENIHW1/VIHW36.QS
5                     None
6                     None
7                     None
8         ENOHW1/VOHW01.QS
9                     None
10                    None
11                    None
12                    None
13                    None
14                    None
15                    None
16                    None
17                    None
18                    None
19                    None
20                    None
21                    None
22                    None
23                    None
24                    None
25                    None
26                    None
27                    None
28                    None
29                    None
                ...       
102026                None
102027                None
102028                None
102029                None
102030                None
102031                None
1

In [38]:
#test_df = data_s1.copy()
#test_df[test_df['link'].apply(lambda x: bool(re.search(r"\$\w+\s+\w+",str(x)))==True)]
test_df = data_ls1.copy()
#test_df[test_df['link'].apply(lambda x: bool(any("$" in s for s in x) and len(x)>1)==True)]
test_df[(test_df['cpu_link']=='PN-D01_P1_$VISW17')]

,chart_block_signal,cpu_link


In [36]:

#report generator of comparison of block in all SD and TDC charts
def report_find_miss_blocks (data_sd,data_tdc, to_file = True, timestamp = datetime.datetime.now()):
    README_HEADER = COMPARATOR_README +"\
\r\nReport: find missing blocks in TDC program\
\r\nDate ant time of report generation: %s \
\r\n===================================================================================\
\r\n"%(str(timestamp))
    
    def generate_report(f):
        print(README_HEADER,file = f) 
        sd_uniq_blocks = data_sd['chartblock'].unique()
        tdc_uniq_blocks = data_tdc['chartblock'].unique()
        for i in sd_uniq_blocks:
            if not i in tdc_uniq_blocks:
                print("Missing block in TDC: ",i,"\r\n",file = f) 
        
        
        
    if to_file:
        file_suffix = "_find_miss_blocks_sd"
        file_prefix = "report_"
        try:
            # Create target Directory
            os.mkdir(os.path.join(PATH_TO_REPORTS))
        except FileExistsError:
            pass
        #sys.stdout = open(os.path.join(PATH_TO_REPORTS,(file_prefix+FILE_NAME_DATA1.split(".")[0]+file_suffix+".txt")), 'w')
        with open(os.path.join(PATH_TO_REPORTS,(file_prefix+FILE_NAME_DATA1.split(".")[0]+file_suffix+".txt")), 'w') as f: 
            generate_report(f)
    else:
        generate_report(sys.stdout)

        #data_a1_1 = data_a1
        #for i, j in data_a1_1.iterrows():
        #    if(not j['out']) and not pd.isnull(j['value']):

    
    
#test_df = data_s2.head(5000).copy()


In [40]:
#############EXECUTION GENERATE REPORT report_find_miss_blocks############
report_find_miss_blocks(data_s2,data_a1,to_file = True)